In [2]:
%load_ext autoreload
%autoreload 2

from torch import nn
from transformers import BertTokenizer, BertForTokenClassification

import torch
import pandas as pd
import numpy as np
import csv
import bert_ner

In [3]:
# convert df to needed format
path = "Train_Tagged_Titles.tsv"
df = pd.read_csv(path, sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE)
df["Record Number"] = df["Record Number"].astype(int)
# forward fill NaN values with last valid value
df = df.ffill()
train_df = bert_ner.convert_df(df)

# put in dataet
tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')
train_dl, labels2id, id2labels, train_ds = bert_ner.create_dl(train_df, df, tokenizer)

In [14]:
# model + weights
model = BertForTokenClassification.from_pretrained( \
    'bert-base-german-cased', num_labels=len(labels2id), id2label=id2labels, label2id=labels2id)
d = torch.load('bert_german_0.p', map_location=torch.device('cpu'))
model.load_state_dict(d['model_state_dict'])

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [20]:
# sample out
decoder = bert_ner.NERDecoder(tokenizer, id2labels)
decoder.decode(train_ds[0])

[('[CLS]', 'O'),
 ('Supreme', 'Modell'),
 ('Nike', 'Marke'),
 ('SB Dunk', 'Produktlinie'),
 ('High', 'Schuhschaft-Typ'),
 ('By any Means', 'Modell'),
 ('Red', 'Farbe'),
 ('US10', 'US-Schuhgröße'),
 ('EU44', 'EU-Schuhgröße'),
 ('Supreme Box', 'No Tag'),
 ('Logo', 'Akzente'),
 ('Air Force', 'Produktlinie'),
 ('[SEP]', 'O')]